# Recommendations

### imports

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
import numpy as np

### bringing in the clean data

In [2]:
thanos = pd.read_csv('../../../data/thanos.csv')
thanos

,steamid,appid,app_title,app_tags,review,fps,voted_up
0,76561198271088129,4000,Garry's Mod,"['Sandbox', 'Multiplayer', 'Funny', 'Moddable'...",good models\n,True,True
1,76561198138206834,4000,Garry's Mod,"['Sandbox', 'Multiplayer', 'Funny', 'Moddable'...",I completely suck at making anything on here a...,True,True
2,76561198128760839,70,Half-Life,"['FPS', 'Sci-fi', 'Action', 'Singleplayer', ""1...",There is not much to say about this old game t...,True,True
3,76561198079636858,6060,"Star Wars: Battlefront 2 (Classic, 2005)","['Action', 'Multiplayer', 'Shooter', 'Third-Pe...",360 noscoped almost everything 10/10 even the ...,True,True
4,76561198238100200,220,Half-Life 2,"['FPS', 'Action', 'Sci-fi', 'Classic', 'Single...",No need for a review. It's practically history.,True,True
...,...,...,...,...,...,...,...
23178,76561198868573076,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",crap game i got 23000 hours and stil silver,True,False
23179,76561198133294233,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...","meh, still so many cheaters",True,False
23180,76561197989220516,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...","to many hackers,high ping servers",True,False
23181,76561198319040167,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...","terrible game, way to many cheaters",True,False


### Making the app_tags column usable with the recommendation system

making the app_tags column

In [3]:
from ast import literal_eval

features = ['app_tags']
for feature in features:
    thanos[feature] = thanos[feature].apply(literal_eval)

In [4]:
thanos

,steamid,appid,app_title,app_tags,review,fps,voted_up
0,76561198271088129,4000,Garry's Mod,"[Sandbox, Multiplayer, Funny, Moddable, Buildi...",good models\n,True,True
1,76561198138206834,4000,Garry's Mod,"[Sandbox, Multiplayer, Funny, Moddable, Buildi...",I completely suck at making anything on here a...,True,True
2,76561198128760839,70,Half-Life,"[FPS, Sci-fi, Action, Singleplayer, 1990's, Sh...",There is not much to say about this old game t...,True,True
3,76561198079636858,6060,"Star Wars: Battlefront 2 (Classic, 2005)","[Action, Multiplayer, Shooter, Third-Person Sh...",360 noscoped almost everything 10/10 even the ...,True,True
4,76561198238100200,220,Half-Life 2,"[FPS, Action, Sci-fi, Classic, Singleplayer, S...",No need for a review. It's practically history.,True,True
...,...,...,...,...,...,...,...
23178,76561198868573076,730,Counter-Strike: Global Offensive,"[FPS, Shooter, Multiplayer, Competitive, Actio...",crap game i got 23000 hours and stil silver,True,False
23179,76561198133294233,730,Counter-Strike: Global Offensive,"[FPS, Shooter, Multiplayer, Competitive, Actio...","meh, still so many cheaters",True,False
23180,76561197989220516,730,Counter-Strike: Global Offensive,"[FPS, Shooter, Multiplayer, Competitive, Actio...","to many hackers,high ping servers",True,False
23181,76561198319040167,730,Counter-Strike: Global Offensive,"[FPS, Shooter, Multiplayer, Competitive, Actio...","terrible game, way to many cheaters",True,False


In [5]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]

In [6]:
for feature in features:
    thanos[feature] = thanos[feature].apply(clean_data)

In [7]:
thanos

,steamid,appid,app_title,app_tags,review,fps,voted_up
0,76561198271088129,4000,Garry's Mod,"[sandbox, multiplayer, funny, moddable, buildi...",good models\n,True,True
1,76561198138206834,4000,Garry's Mod,"[sandbox, multiplayer, funny, moddable, buildi...",I completely suck at making anything on here a...,True,True
2,76561198128760839,70,Half-Life,"[fps, sci-fi, action, singleplayer, 1990's, sh...",There is not much to say about this old game t...,True,True
3,76561198079636858,6060,"Star Wars: Battlefront 2 (Classic, 2005)","[action, multiplayer, shooter, third-personsho...",360 noscoped almost everything 10/10 even the ...,True,True
4,76561198238100200,220,Half-Life 2,"[fps, action, sci-fi, classic, singleplayer, s...",No need for a review. It's practically history.,True,True
...,...,...,...,...,...,...,...
23178,76561198868573076,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...",crap game i got 23000 hours and stil silver,True,False
23179,76561198133294233,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...","meh, still so many cheaters",True,False
23180,76561197989220516,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...","to many hackers,high ping servers",True,False
23181,76561198319040167,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...","terrible game, way to many cheaters",True,False


In [8]:
def create_usable(x):
    return ' '.join(x['app_tags']) + ' ' + ' '

In [9]:
thanos['usable'] = thanos.apply(create_usable, axis=1)

In [10]:
thanos

,steamid,appid,app_title,app_tags,review,fps,voted_up,usable
0,76561198271088129,4000,Garry's Mod,"[sandbox, multiplayer, funny, moddable, buildi...",good models\n,True,True,sandbox multiplayer funny moddable building co...
1,76561198138206834,4000,Garry's Mod,"[sandbox, multiplayer, funny, moddable, buildi...",I completely suck at making anything on here a...,True,True,sandbox multiplayer funny moddable building co...
2,76561198128760839,70,Half-Life,"[fps, sci-fi, action, singleplayer, 1990's, sh...",There is not much to say about this old game t...,True,True,fps sci-fi action singleplayer 1990's shooter ...
3,76561198079636858,6060,"Star Wars: Battlefront 2 (Classic, 2005)","[action, multiplayer, shooter, third-personsho...",360 noscoped almost everything 10/10 even the ...,True,True,action multiplayer shooter third-personshooter...
4,76561198238100200,220,Half-Life 2,"[fps, action, sci-fi, classic, singleplayer, s...",No need for a review. It's practically history.,True,True,fps action sci-fi classic singleplayer storyri...
...,...,...,...,...,...,...,...,...
23178,76561198868573076,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...",crap game i got 23000 hours and stil silver,True,False,fps shooter multiplayer competitive action tea...
23179,76561198133294233,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...","meh, still so many cheaters",True,False,fps shooter multiplayer competitive action tea...
23180,76561197989220516,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...","to many hackers,high ping servers",True,False,fps shooter multiplayer competitive action tea...
23181,76561198319040167,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...","terrible game, way to many cheaters",True,False,fps shooter multiplayer competitive action tea...


In [11]:
title_and_tags = thanos.drop(['steamid', 'appid', 'app_tags', 'review', 'fps', 'voted_up'], axis=1)

In [12]:
title_and_tags

,app_title,usable
0,Garry's Mod,sandbox multiplayer funny moddable building co...
1,Garry's Mod,sandbox multiplayer funny moddable building co...
2,Half-Life,fps sci-fi action singleplayer 1990's shooter ...
3,"Star Wars: Battlefront 2 (Classic, 2005)",action multiplayer shooter third-personshooter...
4,Half-Life 2,fps action sci-fi classic singleplayer storyri...
...,...,...
23178,Counter-Strike: Global Offensive,fps shooter multiplayer competitive action tea...
23179,Counter-Strike: Global Offensive,fps shooter multiplayer competitive action tea...
23180,Counter-Strike: Global Offensive,fps shooter multiplayer competitive action tea...
23181,Counter-Strike: Global Offensive,fps shooter multiplayer competitive action tea...


In [13]:
title_and_tags = title_and_tags.drop_duplicates()

In [14]:
title_and_tags['app_title'].unique()

array(["Garry's Mod", 'Half-Life',
       'Star Wars: Battlefront 2 (Classic, 2005)', 'Half-Life 2',
       'The Ship: Murder Party', 'Fallout: New Vegas', 'Portal 2',
       'Left 4 Dead 2', 'Call of Duty® 4: Modern Warfare®',
       'Borderlands 2', 'Ultimate Doom', 'Half-Life 2: Lost Coast',
       'Counter-Strike: Condition Zero', 'Team Fortress 2',
       'Killing Floor', 'Arma 3', 'S.T.A.L.K.E.R.: Call of Pripyat',
       'Call of Duty® 2', 'Chaser', 'Half-Life: Blue Shift',
       'Red Orchestra 2: Heroes of Stalingrad with Rising Storm',
       'STAR WARS™ Jedi Knight II - Jedi Outcast™', 'BioShock Infinite',
       'STAR WARS™ Jedi Knight - Jedi Academy™',
       'Hitman 2: Silent Assassin', 'Call of Duty®: Modern Warfare® 2',
       "Mirror's Edge™", 'Aliens vs. Predator™',
       'Deus Ex: Game of the Year Edition',
       'Fallout 3: Game of the Year Edition',
       'Arma 2: Operation Arrowhead', 'Crysis 2 - Maximum Edition',
       'ORION: Prelude', 'Crysis', 'Zeno Clash'

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(title_and_tags['usable'])

In [16]:
count_matrix.shape

(166, 204)

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [18]:
recommend = title_and_tags.reset_index(drop=True)
indices = pd.Series(recommend.index, index=recommend['app_title'])

In [19]:
recommend

,app_title,usable
0,Garry's Mod,sandbox multiplayer funny moddable building co...
1,Half-Life,fps sci-fi action singleplayer 1990's shooter ...
2,"Star Wars: Battlefront 2 (Classic, 2005)",action multiplayer shooter third-personshooter...
3,Half-Life 2,fps action sci-fi classic singleplayer storyri...
4,The Ship: Murder Party,multiplayer indie first-person action stealth ...
...,...,...
161,ARMA: Gold Edition,simulation action military strategy fps tactic...
162,Nexuiz,action arenashooter fps multiplayer first-pers...
163,Empires Mod,fps rts shooter strategy first-person 3d team-...
164,Hamilton's Great Adventure,localco-op adventure 3dplatformer puzzle puzzl...


In [20]:
indices

app_title
Garry's Mod                                   0
Half-Life                                     1
Star Wars: Battlefront 2 (Classic, 2005)      2
Half-Life 2                                   3
The Ship: Murder Party                        4
                                           ... 
ARMA: Gold Edition                          161
Nexuiz                                      162
Empires Mod                                 163
Hamilton's Great Adventure                  164
Counter-Strike: Global Offensive            165
Length: 166, dtype: int64

In [42]:
cosine_sim

array([[1.        , 0.40032038, 0.4454354 , ..., 0.25      , 0.35634832,
        0.44907312],
       [0.40032038, 1.        , 0.55634864, ..., 0.32025631, 0.2567763 ,
        0.35300904],
       [0.4454354 , 0.55634864, 1.        , ..., 0.40089186, 0.23809524,
        0.48007936],
       ...,
       [0.25      , 0.32025631, 0.40089186, ..., 1.        , 0.13363062,
        0.53072278],
       [0.35634832, 0.2567763 , 0.23809524, ..., 0.13363062, 1.        ,
        0.21821789],
       [0.44907312, 0.35300904, 0.48007936, ..., 0.53072278, 0.21821789,
        1.        ]])

In [43]:
def get_recommendations(title, score, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]
    print("IDX: ",idx)

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    display(sim_scores)

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #display(sim_scores)

    # Get the scores of the 10 most similar movies
    if score == 0:
        sim_scores = sim_scores[156:167]
    elif score == 1:
        sim_scores = sim_scores[1:11]

    # Get the movie indices
    game_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return recommend['app_title'].iloc[game_indices]

In [44]:
test = get_recommendations("Counter-Strike: Global Offensive", 1)
test = test.to_list()
test

IDX:  165


[(0, 0.4490731195102494),
 (1, 0.3530090432487313),
 (2, 0.4800793585191833),
 (3, 0.2618614682831909),
 (4, 0.31304951684997057),
 (5, 0.21320071635561044),
 (6, 0.23533936216582085),
 (7, 0.5842373946721773),
 (8, 0.5366563145999496),
 (9, 0.4082482904638631),
 (10, 0.3837612894400988),
 (11, 0.21320071635561044),
 (12, 0.5969620057957092),
 (13, 0.6158402871356009),
 (14, 0.5773502691896258),
 (15, 0.7348469228349535),
 (16, 0.25584085962673253),
 (17, 0.5004345937369796),
 (18, 0.2581988897471611),
 (19, 0.21821789023599242),
 (20, 0.6546536707079771),
 (21, 0.2683281572999748),
 (22, 0.17457431218879393),
 (23, 0.2618614682831909),
 (24, 0.2683281572999748),
 (25, 0.6672461249826395),
 (26, 0.13093073414159545),
 (27, 0.44907311951024936),
 (28, 0.21821789023599242),
 (29, 0.21320071635561044),
 (30, 0.6982972487551755),
 (31, 0.21821789023599242),
 (32, 0.5600000000000003),
 (33, 0.2618614682831909),
 (34, 0.14142135623730953),
 (35, 0.5600000000000003),
 (36, 0.13093073414159545

['Arma 3',
 'Arma 2: Operation Arrowhead',
 'Call of Duty®: Modern Warfare® 2',
 'Red Orchestra 2: Heroes of Stalingrad with Rising Storm',
 "Tom Clancy's Rainbow Six® Vegas 2",
 "Tom Clancy's Ghost Recon®",
 'Natural Selection 2',
 'Tribes: Ascend',
 'GROUND BRANCH',
 'ARMA: Gold Edition']

In [34]:
get_recommendations('Counter-Strike: Global Offensive', 1)

15                                                Arma 3
30                           Arma 2: Operation Arrowhead
25                      Call of Duty®: Modern Warfare® 2
20     Red Orchestra 2: Heroes of Stalingrad with Ris...
93                     Tom Clancy's Rainbow Six® Vegas 2
139                            Tom Clancy's Ghost Recon®
74                                   Natural Selection 2
148                                       Tribes: Ascend
64                                         GROUND BRANCH
161                                   ARMA: Gold Edition
Name: app_title, dtype: object

In [24]:
get_recommendations('Half-Life', 0, cosine_sim)

88                          Rogue Warrior
81                            Men of War™
94                ARMA: Combat Operations
159                      Revelations 2012
129         Space Trader: Merchant Marine
87                    Men of War: Vietnam
82              Men of War: Assault Squad
164            Hamilton's Great Adventure
160    Brothers in Arms: Earned in Blood™
13                        Team Fortress 2
Name: app_title, dtype: object

In [25]:
get_recommendations('STAR WARS™ - Dark Forces', 1, cosine_sim)

101    STAR WARS™ Jedi Knight - Mysteries of the Sith™
117              Heretic: Shadow of the Serpent Riders
102             STAR WARS™ Jedi Knight: Dark Forces II
41                                             DOOM II
42                                               QUAKE
47                                         Unreal Gold
1                                            Half-Life
76                                      Red Faction II
152                                 Project: Snowblind
137                             Section 8®: Prejudice™
Name: app_title, dtype: object

In [26]:
get_recommendations('STAR WARS™ - Dark Forces', 0, cosine_sim)

163                           Empires Mod
160    Brothers in Arms: Earned in Blood™
98                             Dino D-Day
81                            Men of War™
164            Hamilton's Great Adventure
87                    Men of War: Vietnam
165      Counter-Strike: Global Offensive
13                        Team Fortress 2
159                      Revelations 2012
82              Men of War: Assault Squad
Name: app_title, dtype: object

In [27]:
get_recommendations('Section 8®: Prejudice™', 1)

108                                        Alpha Prime
78                              Blacklight: Tango Down
140                 Tom Clancy's Rainbow Six Lockdown™
76                                      Red Faction II
152                                 Project: Snowblind
101    STAR WARS™ Jedi Knight - Mysteries of the Sith™
118                                        Red Faction
124                                       Nuclear Dawn
151                                Painkiller Overdose
18                                              Chaser
Name: app_title, dtype: object

In [28]:
get_recommendations('Call of Juarez', 1)

70                Call of Juarez: Bound in Blood
33                                        Crysis
83                                      Far Cry®
21     STAR WARS™ Jedi Knight II - Jedi Outcast™
63                 Far Cry® 2: Fortune's Edition
118                                  Red Faction
96                      Medal of Honor: Airborne
111                              Crysis Warhead®
31                    Crysis 2 - Maximum Edition
107                                    Homefront
Name: app_title, dtype: object

In [29]:
get_recommendations('Medal of Honor™', 0)

131    Pirates, Vikings, and Knights II
141             SiN Episodes: Emergence
10                        Ultimate Doom
113               HeXen: Beyond Heretic
72                                Xotic
112       Carrier Command: Gaea Mission
26                       Mirror's Edge™
75                             Ricochet
164          Hamilton's Great Adventure
129       Space Trader: Merchant Marine
Name: app_title, dtype: object

In [30]:
get_recommendations('Call of Duty®: Modern Warfare® 2', 1)

8      Call of Duty® 4: Modern Warfare®
17                      Call of Duty® 2
65             Call of Duty®: Black Ops
71              Sniper: Ghost Warrior 2
35          Call of Duty®: Black Ops II
107                           Homefront
64                        GROUND BRANCH
155           Frontlines™: Fuel of War™
15                               Arma 3
46              Half-Life 2: Deathmatch
Name: app_title, dtype: object